In [3]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

# from feature_descriptors.sift_descriptor import get_keypoint_and_descriptor
# from feature_matching.flann_matcher import flann_matcher
# from load_h5py_files import load_sift_output, load_flann_output, load_flann_from_images_name
from load_h5py_files import load_matches_from_h5
from batch_matcher import process_batches
from clustering.cluster_images import build_graph, graph_clustering
from batch_descriptor import batch_feature_descriptor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import os
import h5py
import numpy as np
import cv2 as cv
from pathlib import Path
import pandas as pd
import ast

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
from data_preprocess.pseudo_dataset_matching import PseudoMatchingDataset
from torch.utils.data import DataLoader
import torch

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')


pseudo_matching_sift_dataset = PseudoMatchingDataset(pairs_path='data/train_pairs/data_pairs.csv', descriptors_path='evaluation/sift_descriptors_outputs/merged.h5', descriptor_type='sift', device=DEVICE )

train_sift_loader = DataLoader(pseudo_matching_sift_dataset, batch_size=16, shuffle=True, num_workers=10, pin_memory=True)

In [6]:
next(iter(train_sift_loader))

{'descriptor_type': ['sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift'],
 'dataset': ['pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'amy_gardens',
  'pt_stpeters_stpauls',
  'pt_brandenburg_british_buckingham',
  'imc2023_theather_imc2024_church',
  'pt_sacrecoeur_trevi_tajmahal',
  'imc2023_heritage',
  'amy_gardens',
  'pt_piazzasanmarco_grandplace',
  'pt_piazzasanmarco_grandplace',
  'pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'pt_piazzasanmarco_grandplace',
  'pt_stpeters_stpauls',
  'pt_stpeters_stpauls'],
 'image1': ['buckingham_palace_82600926_8760297100.png',
  'outliers_img_8890.png',
  'peach_0097.png',
  'st_pauls_cathedral_72125766_3947493463.png',
  'buckingham_palace_42572558_3847795752.png',
  'church_00047.png',
  'trevi_fountain_54470469_2296508341.png',
  'dioscuri_archive_0004.png',
  'peach_0096.png',
  'piazza_san_marco_2126921

In [3]:
import h5py
from pathlib import Path
from feature_matching.lightglue_matcher import match_features_for_plots
import cv2 as cv
import numpy as np
from batch_matcher import process_batches

# Set up output path
output_dir = Path('evaluation/matches_output')
output_dir.mkdir(exist_ok=True, parents=True)

# Initialize output files for both matchers
flann_sift_output = output_dir / 'sift_flann_matches'


# Run flann matcher
print("\nRunning FLANN matching...")
flann_sift_matches = process_batches(
    train_sift_loader,
    matcher='flann',
    output_path=flann_sift_output
)

print("\nMatching complete!")
print(f"FLANN matches saved to: {flann_sift_output}")

# Print some statistics
print("\nFLANN:")
print(f"Total pairs processed: {len(flann_sift_matches)}")
print(f"Average matches per pair: {np.mean([len(m['matches_idx']) for m in flann_sift_matches]):.1f}")


Running FLANN matching...
Processing 11075 batches with flann...
Batch 1/11075
Batch 2/11075
Batch 3/11075
Batch 4/11075
Batch 5/11075
Batch 6/11075
Batch 7/11075
Batch 8/11075
Batch 9/11075
Batch 10/11075
Saving checkpoint after batch 10
Batch 11/11075
Batch 12/11075
Batch 13/11075
Batch 14/11075
Batch 15/11075
Batch 16/11075
Batch 17/11075
Batch 18/11075
Batch 19/11075
Batch 20/11075
Saving checkpoint after batch 20
Batch 21/11075
Batch 22/11075
Batch 23/11075
Batch 24/11075
Batch 25/11075
Batch 26/11075
Batch 27/11075
Batch 28/11075
Batch 29/11075
Batch 30/11075
Saving checkpoint after batch 30
Batch 31/11075
Batch 32/11075
Batch 33/11075
Batch 34/11075
Batch 35/11075
Batch 36/11075
Batch 37/11075
Batch 38/11075
Batch 39/11075
Batch 40/11075
Saving checkpoint after batch 40
Batch 41/11075
Batch 42/11075
Batch 43/11075
Batch 44/11075
Batch 45/11075
Batch 46/11075
Batch 47/11075
Batch 48/11075
Batch 49/11075
Batch 50/11075
Saving checkpoint after batch 50
Batch 51/11075
Batch 52/1107

/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [4]:
with h5py.File(flann_sift_output / "11070_flann_sift.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key]
        print(f"{key}: {num_rows} rows")

    print(len(f.keys()))

0: <HDF5 group "/0" (9 members)> rows
1: <HDF5 group "/1" (9 members)> rows
10: <HDF5 group "/10" (9 members)> rows
100: <HDF5 group "/100" (9 members)> rows
101: <HDF5 group "/101" (9 members)> rows
102: <HDF5 group "/102" (9 members)> rows
103: <HDF5 group "/103" (9 members)> rows
104: <HDF5 group "/104" (9 members)> rows
105: <HDF5 group "/105" (9 members)> rows
106: <HDF5 group "/106" (9 members)> rows
107: <HDF5 group "/107" (9 members)> rows
108: <HDF5 group "/108" (9 members)> rows
109: <HDF5 group "/109" (9 members)> rows
11: <HDF5 group "/11" (9 members)> rows
110: <HDF5 group "/110" (9 members)> rows
111: <HDF5 group "/111" (9 members)> rows
112: <HDF5 group "/112" (9 members)> rows
113: <HDF5 group "/113" (9 members)> rows
114: <HDF5 group "/114" (9 members)> rows
115: <HDF5 group "/115" (9 members)> rows
116: <HDF5 group "/116" (9 members)> rows
117: <HDF5 group "/117" (9 members)> rows
118: <HDF5 group "/118" (9 members)> rows
119: <HDF5 group "/119" (9 members)> rows
12: 

In [5]:
with h5py.File(flann_sift_output / "11075_flann_sift.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key]
        print(f"{key}: {num_rows} rows")

    print(len(f.keys()))

0: <HDF5 group "/0" (9 members)> rows
1: <HDF5 group "/1" (9 members)> rows
10: <HDF5 group "/10" (9 members)> rows
11: <HDF5 group "/11" (9 members)> rows
12: <HDF5 group "/12" (9 members)> rows
13: <HDF5 group "/13" (9 members)> rows
14: <HDF5 group "/14" (9 members)> rows
15: <HDF5 group "/15" (9 members)> rows
16: <HDF5 group "/16" (9 members)> rows
17: <HDF5 group "/17" (9 members)> rows
18: <HDF5 group "/18" (9 members)> rows
19: <HDF5 group "/19" (9 members)> rows
2: <HDF5 group "/2" (9 members)> rows
20: <HDF5 group "/20" (9 members)> rows
21: <HDF5 group "/21" (9 members)> rows
22: <HDF5 group "/22" (9 members)> rows
23: <HDF5 group "/23" (9 members)> rows
24: <HDF5 group "/24" (9 members)> rows
25: <HDF5 group "/25" (9 members)> rows
26: <HDF5 group "/26" (9 members)> rows
27: <HDF5 group "/27" (9 members)> rows
28: <HDF5 group "/28" (9 members)> rows
29: <HDF5 group "/29" (9 members)> rows
3: <HDF5 group "/3" (9 members)> rows
30: <HDF5 group "/30" (9 members)> rows
31: <HDF

In [8]:
from load_h5py_files import load_matches_from_h5

# Set up output path
output_dir = Path('evaluation/matches_output')
output_dir.mkdir(exist_ok=True, parents=True)

# Initialize output files for both matchers
flann_sift_output = output_dir / 'sift_flann_matches'
load_matches_from_h5(flann_sift_output / "20_flann_sift.h5")

[{'image1': 'vineyard_split_2_frame_1285.png',
  'image2': 'vineyard_split_2_frame_1215.png',
  'dataset': 'fbk_vineyard',
  'points0': tensor([[333.0617, 555.6654],
          [336.4977, 826.5097],
          [587.3848, 830.1970],
          [609.7038, 203.0695],
          [506.1097, 660.3701]]),
  'points1': tensor([[389.9074, 830.5649],
          [640.9841, 793.1812],
          [689.1013, 533.2594],
          [593.4620, 219.6947],
          [504.3139, 647.0392]]),
  'features1': {'scene': 'vineyard_split_2'},
  'features2': {'scene': 'vineyard_split_2'},
  'matches_idx': tensor([[  25,  530],
          [  66,  389],
          [ 249,   64],
          [ 430,  689],
          [ 434, 1245]]),
  'distances': tensor([0.2507, 0.2814, 0.2790, 0.2387, 0.1026])},
 {'image1': 'piazza_san_marco_16953110_11565839383.png',
  'image2': 'grand_place_brussels_12650889_5688674640.png',
  'dataset': 'pt_piazzasanmarco_grandplace',
  'points0': tensor([[999.5556, 189.6105],
          [833.5601, 347.2261],

## with disk

In [7]:
from data_preprocess.pseudo_dataset_matching import PseudoMatchingDataset
from torch.utils.data import DataLoader
import torch

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')


pseudo_matching_disk_dataset = PseudoMatchingDataset(pairs_path='data/train_pairs/data_pairs.csv', descriptors_path='evaluation/disk_descriptors_outputs/merged.h5', descriptor_type='disk', device=DEVICE)

train_disk_loader = DataLoader(pseudo_matching_disk_dataset, batch_size=16, shuffle=True, num_workers=10, pin_memory=True)

In [5]:
next(iter(train_loader))

{'descriptor_type': ['disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk'],
 'dataset': ['imc2023_heritage',
  'pt_sacrecoeur_trevi_tajmahal',
  'pt_brandenburg_british_buckingham',
  'imc2024_lizard_pond',
  'imc2024_lizard_pond',
  'imc2024_dioscuri_baalshamin',
  'imc2023_heritage',
  'pt_brandenburg_british_buckingham',
  'imc2024_lizard_pond',
  'amy_gardens',
  'pt_brandenburg_british_buckingham',
  'pt_stpeters_stpauls',
  'pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'pt_stpeters_stpauls',
  'imc2023_heritage'],
 'image1': ['outliers_img_8662.png',
  'trevi_fountain_59883665_3581996971.png',
  'buckingham_palace_82600926_8760297100.png',
  'outliers_img_20230617_181852.png',
  'pond_00440.png',
  'baalshamin_19739156806_e4f69d8792_o.png',
  'dioscuri_img_0348.png',
  'british_museum_67471600_1224757581.png',
  'lizard_00618.png',
  'peach_0116.png',
  'british_

In [8]:
import h5py
from pathlib import Path
from feature_matching.lightglue_matcher import match_features_for_plots
import cv2 as cv
import numpy as np
from batch_matcher import process_batches

# Set up output path
output_dir = Path('evaluation/matches_output')
output_dir.mkdir(exist_ok=True, parents=True)

# Initialize output files for both matchers
flann_disk_output = output_dir / 'disk_flann_matches'


# Run flann matcher
print("\nRunning FLANN matching...")
flann_disk_matches = process_batches(
    train_disk_loader,
    matcher='flann',
    output_path=flann_disk_output
)

print("\nMatching complete!")
print(f"FLANN matches saved to: {flann_disk_output}")

# Print some statistics
print("\nFLANN:")
print(f"Total pairs processed: {len(flann_disk_matches)}")
print(f"Average matches per pair: {np.mean([len(m['matches_idx']) for m in flann_disk_matches]):.1f}")


Running FLANN matching...
Processing 11075 batches with flann...
Batch 1/11075
Batch 2/11075
Batch 3/11075
Batch 4/11075
Batch 5/11075
Batch 6/11075
Batch 7/11075
Batch 8/11075
Batch 9/11075
Batch 10/11075
Saving checkpoint after batch 10
Batch 11/11075
Batch 12/11075
Batch 13/11075
Batch 14/11075
Batch 15/11075
Batch 16/11075
Batch 17/11075
Batch 18/11075
Batch 19/11075
Batch 20/11075
Saving checkpoint after batch 20
Batch 21/11075
Batch 22/11075
Batch 23/11075
Batch 24/11075
Batch 25/11075
Batch 26/11075
Batch 27/11075
Batch 28/11075
Batch 29/11075
Batch 30/11075
Saving checkpoint after batch 30
Batch 31/11075
Batch 32/11075
Batch 33/11075
Batch 34/11075
Batch 35/11075
Batch 36/11075
Batch 37/11075
Batch 38/11075
Batch 39/11075
Batch 40/11075
Saving checkpoint after batch 40
Batch 41/11075
Batch 42/11075
Batch 43/11075
Batch 44/11075
Batch 45/11075
Batch 46/11075
Batch 47/11075
Batch 48/11075
Batch 49/11075
Batch 50/11075
Saving checkpoint after batch 50
Batch 51/11075
Batch 52/1107

/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
from load_h5py_files import load_matches_from_h5

load_matches_from_h5(flann_disk_output / "10_flann_disk.h5")

[{'image1': 'vineyard_split_2_frame_1165.png',
  'image2': 'vineyard_split_3_frame_0200.png',
  'dataset': 'fbk_vineyard',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([], size=(0, 2)),
  'features1': {'scene': 'vineyard_split_2'},
  'features2': {'scene': 'vineyard_split_3'},
  'matches_idx': tensor([], size=(0, 2), dtype=torch.int64),
  'distances': tensor([])},
 {'image1': 'trevi_fountain_01241225_4293371290.png',
  'image2': 'sacre_coeur_41224690_4580747259.png',
  'dataset': 'pt_sacrecoeur_trevi_tajmahal',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([], size=(0, 2)),
  'features1': {'scene': 'trevi_fountain'},
  'features2': {'scene': 'sacre_coeur'},
  'matches_idx': tensor([], size=(0, 2), dtype=torch.int64),
  'distances': tensor([])},
 {'image1': 'piazza_san_marco_49865216_1748767363.png',
  'image2': 'grand_place_brussels_36406299_8032284977.png',
  'dataset': 'pt_piazzasanmarco_grandplace',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([

## create csv for the result

In [29]:
from data_preprocess.matcher_csv import MatcherCsv

MatcherCsv("flann", "sift", flann_sift_output, "csv_matcher")
MatcherCsv("flann", "disk", flann_disk_output, "csv_matcher")

In [31]:
df_flann_sift = pd.read_csv("csv_matcher/result_flann_sift.csv")
df_flann_sift

,image1,image2,dataset,points0,points1,features1,features2,matches_idx,distances
0,trevi_fountain_29250228_5522743340.png,sacre_coeur_16776129_4239663609.png,pt_sacrecoeur_trevi_tajmahal,"[[340.9080505371094, 797.3765258789062]]","[[771.86669921875, 693.5773315429688]]",{'scene': 'trevi_fountain'},{'scene': 'sacre_coeur'},"[[1322, 1309]]",[0.40538719296455383]
1,trevi_fountain_90077341_5746101059.png,sacre_coeur_39172134_3155998144.png,pt_sacrecoeur_trevi_tajmahal,"[[773.9786987304688, 882.8646850585938]]","[[316.791015625, 569.4338989257812]]",{'scene': 'trevi_fountain'},{'scene': 'sacre_coeur'},"[[1533, 43]]",[0.4005594849586487]
2,st_pauls_cathedral_61928556_183808445.png,st_peters_square_40696807_2666761892.png,pt_stpeters_stpauls,"[[298.014892578125, 582.785888671875], [711.53...","[[609.8031005859375, 473.7809753417969], [850....",{'scene': 'st_pauls_cathedral'},{'scene': 'st_peters_square'},"[[373, 508], [813, 818], [897, 635], [1222, 15...","[0.3130742609500885, 0.3354966342449188, 0.399..."
3,british_museum_90026426_4349430506.png,buckingham_palace_37524858_3739802247.png,pt_brandenburg_british_buckingham,"[[1008.3880615234375, 808.1184692382812], [411...","[[613.0118408203125, 759.8627319335938], [757....",{'scene': 'british_museum'},{'scene': 'buckingham_palace'},"[[48, 1065], [326, 639], [485, 1080], [981, 31]]","[0.3602631092071533, 0.2795352041721344, 0.352..."
4,dioscuri_img_0424.png,cyprus_dsc_6500.png,imc2023_heritage,"[[845.1094970703125, 599.3341064453125], [354....","[[959.1676635742188, 406.8739318847656], [452....",{'scene': 'dioscuri'},{'scene': 'cyprus'},"[[1086, 1602], [1125, 1274], [1223, 1602], [14...","[0.3123491704463959, 0.27762356400489807, 0.29..."
...,...,...,...,...,...,...,...,...,...
177193,cyprus_dsc_6569.png,cyprus_dsc_6597.png,imc2023_heritage,"[[649.0734252929688, 30.52924156188965], [641....","[[683.790283203125, 353.63116455078125], [654....",{'scene': 'cyprus'},{'scene': 'cyprus'},"[[574, 1198], [600, 505], [637, 829], [652, 19...","[0.216144397854805, 0.2690103352069855, 0.1513..."
177194,piazza_san_marco_62044241_3006469222.png,piazza_san_marco_66788909_420268870.png,pt_piazzasanmarco_grandplace,"[[14.141103744506836, 488.81103515625], [463.1...","[[497.5968933105469, 253.1563720703125], [630....",{'scene': 'piazza_san_marco'},{'scene': 'piazza_san_marco'},"[[81, 1344], [245, 1269], [1350, 1512], [1413,...","[0.31805673241615295, 0.3613090217113495, 0.38..."
177195,outliers_img_8665_thumb.png,dioscuri_archive_0116.png,imc2023_heritage,"[[272.5780334472656, 583.171630859375]]","[[171.30392456054688, 975.4847412109375]]",{'scene': 'outliers'},{'scene': 'dioscuri'},"[[96, 191]]",[0.3858760893344879]
177196,piazza_san_marco_94011496_2894625345.png,grand_place_brussels_33234837_3550291978.png,pt_piazzasanmarco_grandplace,"[[886.5260620117188, 854.91357421875], [851.55...","[[871.47900390625, 848.4865112304688], [801.55...",{'scene': 'piazza_san_marco'},{'scene': 'grand_place_brussels'},"[[280, 70], [1514, 1096]]","[0.2092025876045227, 0.36995160579681396]"


In [35]:
# example to convert list data / dict data from string to list / dict
ast.literal_eval(df_flann_sift["points0"][2])

[[298.014892578125, 582.785888671875],
 [711.53173828125, 595.067626953125],
 [700.3175659179688, 412.0234375],
 [677.4392700195312, 339.7024230957031],
 [727.6646728515625, 635.3369750976562]]

In [37]:
ast.literal_eval(df_flann_sift["features1"][2])

{'scene': 'st_pauls_cathedral'}

In [32]:
df_flann_disk = pd.read_csv("csv_matcher/result_flann_disk.csv")
df_flann_disk

,image1,image2,dataset,points0,points1,features1,features2,matches_idx,distances
0,vineyard_split_2_frame_1165.png,vineyard_split_3_frame_0200.png,fbk_vineyard,[],[],{'scene': 'vineyard_split_2'},{'scene': 'vineyard_split_3'},[],[]
1,trevi_fountain_01241225_4293371290.png,sacre_coeur_41224690_4580747259.png,pt_sacrecoeur_trevi_tajmahal,[],[],{'scene': 'trevi_fountain'},{'scene': 'sacre_coeur'},[],[]
2,piazza_san_marco_49865216_1748767363.png,grand_place_brussels_36406299_8032284977.png,pt_piazzasanmarco_grandplace,[],[],{'scene': 'piazza_san_marco'},{'scene': 'grand_place_brussels'},[],[]
3,lizard_00585.png,lizard_00592.png,imc2024_lizard_pond,[],[],{'scene': 'lizard'},{'scene': 'lizard'},[],[]
4,lizard_00169.png,pond_00181.png,imc2024_lizard_pond,[],[],{'scene': 'lizard'},{'scene': 'pond'},[],[]
...,...,...,...,...,...,...,...,...,...
177193,sacre_coeur_73084286_312927075.png,sacre_coeur_02928139_3448003521.png,pt_sacrecoeur_trevi_tajmahal,[],[],{'scene': 'sacre_coeur'},{'scene': 'sacre_coeur'},[],[]
177194,trevi_fountain_81409989_5495330211.png,taj_mahal_48590156_9372591744.png,pt_sacrecoeur_trevi_tajmahal,[],[],{'scene': 'trevi_fountain'},{'scene': 'taj_mahal'},[],[]
177195,brandenburg_gate_43929418_4177129462.png,brandenburg_gate_82818583_7062231077.png,pt_brandenburg_british_buckingham,"[[593.0, 386.0], [497.0, 400.0], [533.0, 409.0...","[[543.0, 207.0], [488.0, 211.0], [509.0, 218.0...",{'scene': 'brandenburg_gate'},{'scene': 'brandenburg_gate'},"[[231, 81], [345, 102], [429, 134], [430, 138]...","[0.6907773613929749, 0.6593204140663147, 0.664..."
177196,dioscuri_img_0376.png,cyprus_dsc_6597.png,imc2023_heritage,[],[],{'scene': 'dioscuri'},{'scene': 'cyprus'},[],[]


In [40]:
ast.literal_eval(df_flann_sift["points0"][177197])

[[147.95132446289062, 761.0935668945312],
 [350.77447509765625, 248.248291015625],
 [309.78204345703125, 674.9468383789062],
 [308.12896728515625, 308.25714111328125],
 [725.9365234375, 565.4805908203125],
 [659.6013793945312, 563.5665893554688]]

In [41]:
ast.literal_eval(df_flann_sift["points1"][177197])

[[956.0643920898438, 356.9911804199219],
 [519.61474609375, 342.6565246582031],
 [559.3860473632812, 627.1792602539062],
 [199.01739501953125, 373.8225402832031],
 [634.3848266601562, 378.4701843261719],
 [884.5348510742188, 428.4748229980469]]

In [42]:
ast.literal_eval(df_flann_sift["features1"][177197])

{'scene': 'st_pauls_cathedral'}